In [1]:
# Compiled with mkl.
# To install the _exact_ same version, run:
# pip install mxnet-mkl==1.3.0b20180711
import mxnet as mx
import numpy as np
from mxnet import nd, autograd, gluon
import multiprocessing

print(mx.__version__)
print(np.__version__)

1.3.0
1.14.5


# Creating the Network

In [ ]:
mnist = mx.test_utils.get_mnist()
mx.random.seed(1)
ctx = mx.cpu(0)
batch_size = 64
num_inputs = 784
num_outputs = 10
data_iter = mx.io.NDArrayIter(mnist['train_data'], mnist['train_label'], batch_size)
num_fc = 512

net = gluon.nn.HybridSequential()
with net.name_scope():
    net.add(gluon.nn.Conv2D(channels=8, kernel_size=5, activation='relu'))
    net.add(gluon.nn.MaxPool2D(pool_size=2, strides=2))
    net.add(gluon.nn.Conv2D(channels=16, kernel_size=5, activation='relu'))
    net.add(gluon.nn.MaxPool2D(pool_size=2, strides=2))
    net.add(gluon.nn.Flatten())
    net.add(gluon.nn.Dense(num_fc, activation="relu"))
    net.add(gluon.nn.Dense(num_outputs))
net.hybridize()
# Parameter initialization
net.collect_params().initialize(mx.init.Xavier(magnitude=2.24), ctx=ctx)
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': .1})
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()
for i, batch in enumerate(data_iter):
    data = batch.data[0].as_in_context(ctx)
    label = batch.label[0].as_in_context(ctx)
    with autograd.record():
        output = net(data)
        loss = softmax_cross_entropy(output, label)
    loss.backward()
    trainer.step(data.shape[0])
data_iter.reset()

# Listing the Internal Names

In [7]:
data = mx.sym.Variable('data')
sym = net(data)

import pprint
pprint.pprint(sym.get_internals().list_outputs())

['data',
 'hybridsequential0_conv0_weight',
 'hybridsequential0_conv0_bias',
 'hybridsequential0_conv0_fwd_output',
 'hybridsequential0_conv0_relu_fwd_output',
 'hybridsequential0_pool0_fwd_output',
 'hybridsequential0_conv1_weight',
 'hybridsequential0_conv1_bias',
 'hybridsequential0_conv1_fwd_output',
 'hybridsequential0_conv1_relu_fwd_output',
 'hybridsequential0_pool1_fwd_output',
 'hybridsequential0_flatten0_reshape0_output',
 'hybridsequential0_dense0_weight',
 'hybridsequential0_dense0_bias',
 'hybridsequential0_dense0_fwd_output',
 'hybridsequential0_dense0_relu_fwd_output',
 'hybridsequential0_dense1_weight',
 'hybridsequential0_dense1_bias',
 'hybridsequential0_dense1_fwd_output']


# Change Blocks to Symbols (And checking that it still works)

In [5]:
def block2symbol(block):
    data = mx.sym.Variable('data')
    sym = block(data)
    params = block.collect_params()
    arg_params = {k : mx.nd.array(v.data().asnumpy()) for k, v in params.items()}
    aux_params = {k : mx.nd.array(v.data().asnumpy()) for k, v in params.items()}
    return sym, arg_params, aux_params

# Converting gluon into module
mx_sym, args, auxs = block2symbol(net)
# Need name = softmax so that label_names can handle softmax_label
mx_sym = mx.sym.SoftmaxOutput(data=mx_sym, name='softmax')
model = mx.mod.Module(symbol = mx_sym, context = mx.cpu(), 
                       label_names = ['softmax_label'])
model.bind(for_training=False,
           data_shapes = data_iter.provide_data, 
           label_shapes = data_iter.provide_label)
model.set_params(args, auxs)
labels = mnist['train_label'][:50]
data_iter.reset()
print(np.argmax(model.predict(data_iter)[:50].asnumpy(), axis=1))
print(labels)

[5 0 4 1 9 2 1 3 1 4 3 5 3 6 1 7 2 8 6 9 4 0 9 1 1 2 4 3 2 7 3 8 6 9 0 5 6
 0 7 6 1 8 7 9 3 9 8 5 5 3]
[5 0 4 1 9 2 1 3 1 4 3 5 3 6 1 7 2 8 6 9 4 0 9 1 1 2 4 3 2 7 3 8 6 9 0 5 6
 0 7 6 1 8 7 9 3 9 8 5 9 3]


# Quantizing Parameters

In [6]:
# Excluding conv0: mkldnn cannot handle channels % 4 != 0.

# Excluding everything else: Apparently some ops are not registered:
# Upon bind:
# [19:24:33] src/executor/attach_op_execs_pass.cc:336: Neither FCompute nor FComputeEx registered _contrib_quantized_fully_connected
# [19:24:33] src/executor/attach_op_execs_pass.cc:336: Neither FCompute nor FComputeEx registered _contrib_quantized_fully_connected
# Segmentation fault: 11
# Stack trace returned 10 entries:
# [bt] (0) 0   libmxnet.so                         0x000000010a1066d4 libmxnet.so + 26324
# [bt] (1) 1   libmxnet.so                         0x000000010b78d836 MXTVMBridge + 3974902
# [bt] (2) 2   libsystem_platform.dylib            0x00000001018d9f5a _sigtramp + 26
# [bt] (3) 3   ???                                 0x0000000000000002 0x0 + 2
# [bt] (4) 4   libmxnet.so                         0x000000010b23592c MXNDListFree + 146732
# [bt] (5) 5   libmxnet.so                         0x000000010b241803 MXNDListFree + 195587
# [bt] (6) 6   libmxnet.so                         0x000000010b24cf54 MXNDListFree + 242516
# [bt] (7) 7   libmxnet.so                         0x000000010b25210a MXNDListFree + 263434
# [bt] (8) 8   libmxnet.so                         0x000000010b1dcad0 MXExecutorSimpleBind + 8656
# [bt] (9) 9   _ctypes.cpython-36m-darwin.so       0x000000010767fd17 ffi_call_unix64 + 79
qsym, qarg_params, qaux_params = mx.contrib.quantization.quantize_model(
        sym=model._symbol, 
        arg_params=model._arg_params, 
        aux_params=model._aux_params,
        excluded_sym_names=['hybridsequential0_conv0_fwd',
                           'hybridsequential0_dense0_fwd',
                           'hybridsequential0_dense1_fwd',
                          ],
        ctx=ctx, 
        calib_data=data_iter,
    )

In [8]:
import pprint
pprint.pprint(qsym.get_internals().list_outputs())

NameError: name 'qsym' is not defined

In [8]:
for k in qarg_params:
    print("dtype of {} : {}".format(k, qarg_params[k].dtype))

dtype of hybridsequential0_conv0_weight : <class 'numpy.float32'>
dtype of hybridsequential0_conv0_bias : <class 'numpy.float32'>
dtype of hybridsequential0_conv1_weight_quantize : <class 'numpy.int8'>
dtype of hybridsequential0_conv1_weight_quantize_min : <class 'numpy.float32'>
dtype of hybridsequential0_conv1_weight_quantize_max : <class 'numpy.float32'>
dtype of hybridsequential0_conv1_bias_quantize : <class 'numpy.int8'>
dtype of hybridsequential0_conv1_bias_quantize_min : <class 'numpy.float32'>
dtype of hybridsequential0_conv1_bias_quantize_max : <class 'numpy.float32'>
dtype of hybridsequential0_dense0_weight : <class 'numpy.float32'>
dtype of hybridsequential0_dense0_bias : <class 'numpy.float32'>
dtype of hybridsequential0_dense1_weight : <class 'numpy.float32'>
dtype of hybridsequential0_dense1_bias : <class 'numpy.float32'>


# Creating the Module with Quantized Symbols

In [9]:
qmodel = mx.mod.Module(symbol = qsym, context = mx.cpu())

# Preparing to `predict`

## Binding with same `data_iter` as before

In [10]:
qmodel.bind(for_training=False,
           data_shapes = data_iter.provide_data, 
           label_shapes = data_iter.provide_label)

## Setting parameters to quantized params

In [12]:
qmodel.set_params(qarg_params, qaux_params)

# Trying to predict gives an error...

In [13]:
qmodel.predict(data_iter)

MXNetError: [19:31:04] src/operator/quantization/mkldnn/mkldnn_quantized_conv.cc:41: Check failed: in_data[0].dtype() == mshadow::kUint8 (5 vs. 3) mkldnn_quantized_conv op only supports uint8 as input type

Stack trace returned 10 entries:
[bt] (0) 0   libmxnet.so                         0x0000000109a056d4 libmxnet.so + 26324
[bt] (1) 1   libmxnet.so                         0x0000000109a0548f libmxnet.so + 25743
[bt] (2) 2   libmxnet.so                         0x0000000109a3a093 libmxnet.so + 241811
[bt] (3) 3   libmxnet.so                         0x000000010ab57745 MXNDListFree + 289605
[bt] (4) 4   libmxnet.so                         0x000000010ab233fc MXNDListFree + 75772
[bt] (5) 5   libmxnet.so                         0x000000010ab26931 MXNDListFree + 89393
[bt] (6) 6   libmxnet.so                         0x000000010ab26847 MXNDListFree + 89159
[bt] (7) 7   libmxnet.so                         0x000000010ab24175 MXNDListFree + 79221
[bt] (8) 8   libsystem_pthread.dylib             